In [0]:
#import packages
import urllib.request as urllib
import pandas as pd
from bs4 import BeautifulSoup

#String -> List<String>
#Finds all of the links to schools on the search page 
def find_link(script_text):
  profilelist = script_text.split('"profile"')
  result = []
  for profile in profilelist:
    temp = profile[2:]
    link = temp.split('"')[0]
    if link != "cript type=":
      result.append(link)
  return result

#pull our graduation rate data from within the string
def find_gr(soup): 
  posn1=str(soup.findAll("script")).find("4-year high school graduation rate")
  if posn1 == -1:
    gr = "Not available"
  else:
    posn2=str(soup.findAll("script")).find("4-year high school graduation rate", posn1 + 1)
    posn3=str(soup.findAll("script")).find("4-year high school graduation rate", posn2 + 1)
    gr =str(soup.findAll("script"))[posn3-17:posn3-14]
    if not isinstance(gr[0],int):
      gr = gr[1:]
    elif not isinstance(gr[1], int):
      gr = gr[2]  
  print(gr)
  return gr

#pull SAT data from within the javascript string
def findSAT(soup):
  ind=str(soup.findAll("script")).rfind("Average SAT score")
  if ind == -1:
    SAT = "Not available"
  else:
    SAT = str(soup.findAll("script"))[ind-len("Average SAT score")-15:ind-len("Average SAT Score")-10]
  print("SAT: " + SAT)
  return SAT


#pull AP data from within the javascript string
def find_AP(soup):
  posn1=str(soup.findAll("script")).find("AP course participation")
  if posn1 == -1:
    AP = "Not available"
  else:
    posn2=str(soup.findAll("script")).find("AP course participation", posn1 + 1)
    AP= str(soup.findAll("script"))[posn2-16:posn2-14]
  return AP

#pull college attendance data from within the javascript string
def find_collegeatt(soup):
  ind=str(soup.findAll("script")).rfind("Graduates pursuing college or vocational program")
  if ind == -1:
    college_att = "Not available"
  else:
    college_att = str(soup.findAll("script"))[ind-len("Graduates pursuing college or vocational program")+20:ind-len("Graduates pursuing college or vocational program")+22]
  return college_att

#pull college retention data from within the javascript string
def find_collegeret(soup):
  ind=str(soup.findAll("script")).rfind("Graduates who returned to college for a second year")
  if ind == -1:
    college_ret = "Not available"
  else:
    college_ret = str(soup.findAll("script"))[ind-len("Graduates who returned to college for a second year")+23:ind-len("Graduates who returned to college for a second year")+25]
  return college_ret

#creates a temporary dictionary from each of our school links
def make_temp_dict(link, state):
  result = {}
  soup = BeautifulSoup(urllib.urlopen("https://www.greatschools.org" + link).read())
  result["State"] = state
  result["Name"] = soup.find(class_='school-name').get_text()
  print("Name: " + result["Name"])
  result["Link"] = link
  result["Address"] = soup.find(class_='content').get_text()
  result["Graduation Rate"] = find_gr(soup)
  result["SAT"] = findSAT(soup)
  result["AP"] = find_AP(soup)
  result["College Attendance"] = find_collegeatt(soup)
  result["College Retention"] = find_collegeret(soup)
  return result

#scrapes data for the new england states
df_schools = pd.DataFrame(columns = ["State", "Name", "Link", "Address", "Graduation Rate", "SAT", "AP", "College Attendance", "College Retention"])

#runs through each state
for bloop in [("massachusetts", 15), ("maine", 7), ("vermont", 4), ("new-hampshire",4) , ("rhode-island",3), ("connecticut",12)]:
  state = bloop[0]
  page = bloop[1]
  for pagenum in range(page):
    html = urllib.urlopen('https://www.greatschools.org/' + state + '/schools/?gradeLevels=h&page=' + str(pagenum) + '&st%5B%5D=public_charter&st%5B%5D=public&view=table')
    soup = BeautifulSoup(html.read())
    script = soup.find("script")
    for link in find_link(str(script)):
      df_schools = df_schools.append(make_temp_dict(link, state), ignore_index=True)
print(df_schools.head())





Name: MA Academy for Math and Science School
00
SAT: 1446"
Name: Lynnfield High School
99
SAT: 1141"
Name: Acton-Boxborough Regional High School
97
SAT: 1307"
Name: Ashland High School
96
SAT: 1200"
Name: Blackstone Valley Regional Vocational Tech High School
99
SAT: 1091"
Name: Boston Latin School
97
SAT: 1310"
Name: Chelmsford High School
98
SAT: 1182"
Name: Concord Carlisle High School
97
SAT: 1267"
Name: Dover-Sherborn Regional High School
99
SAT: 1285"
Name: Duxbury High School
99
SAT: 1181"
Name: Hamilton-Wenham Regional High School
99
SAT: 1177"
Name: Hingham High School
97
SAT: 1201"
Name: Hopkinton High School
99
SAT: 1246"
Name: Lexington High School
99
SAT: 1297"
Name: Longmeadow High School
99
SAT: 1197"
Name: Masconomet Regional High School
98
SAT: 1179"
Name: Medfield Senior High School
98
SAT: 1208"
Name: Needham High School
98
SAT: 1218"
Name: Algonquin Regional High School
98
SAT: 1191"
Name: Norwell High School
00
SAT: 1176"
Name: Wellesley Sr High School
97
SAT: 1224

In [0]:
import urllib.request as urllib
import pandas as pd
from bs4 import BeautifulSoup 

#creates a dataframe for our college info
df_colleges = pd.DataFrame(columns = ["College", "City"])

#crawls and scrapes college name and city
soup = BeautifulSoup(urllib.urlopen('https://nces.ed.gov/collegenavigator/?s=CT+ME+MA+NH+RI+VT&ic=1&pg=0').read())
for link in soup.find_all("strong"):
  if link.text =="U.S. Department of Education" or link.text =="13" or link.text =="12" or link.text == "11" or link.text =="10" or link.text =="1" or link.text =="2" or link.text =="3" or link.text =="4" or link.text =="5" or link.text =="6" or link.text =="7" or link.text =="8" or link.text =="9" or link.text == "My Favorites" or link.text == "comparisons":
    pass
  else:
    tempdict= {"College": link.text, "City": link.parent.parent.text[len(link.text)::] }
    df_colleges = df_colleges.append(tempdict, ignore_index=True)

In [0]:
pd.set_option('display.max_rows', None)

#we decided to only use one college per city to decrease the number of API calls we would need 
#as well as to make our results more meaningful- for example
#the difference in distance from BLS to BU vs Northeastern is likely negligible, what's more important is that it's close to universities
df_colleges_unique = df_colleges.copy()
seen_places = []
del_indexes = []
count = -1
for row in df_colleges_unique["City"]:
  count = count + 1
  if row not in seen_places:
    seen_places.append(row)
  else: 
    del_indexes.append(count)
for index in del_indexes:
  df_colleges_unique = df_colleges_unique.drop(index)
df_colleges_unique

,College,City
0,Albertus Magnus College,"New Haven, Connecticut"
1,American International College,"Springfield, Massachusetts"
2,Amherst College,"Amherst, Massachusetts"
3,Anna Maria College,"Paxton, Massachusetts"
4,Antioch University-New England,"Keene, New Hampshire"
5,Assumption College,"Worcester, Massachusetts"
6,Babson College,"Wellesley, Massachusetts"
7,Bard College at Simon's Rock,"Great Barrington, Massachusetts"
8,Bates College,"Lewiston, Maine"
9,Bay Path University,"Longmeadow, Massachusetts"


In [0]:
df_schools.to_csv ('df_schools.csv', header=True)


In [0]:
df_MA = pd.read_csv('massachusetts.csv')
df_VT = pd.read_csv('vermont.csv')
df_CT = pd.read_csv('connecticut.csv')
df_RI = pd.read_csv('rhode_island.csv')


HS_MA = ""
HS_VT = ""
HS_CT = ""
HS_RI = ""

#creates strings separated by pipes because that's how the distance matrix api works

for ad in df_MA["Address"]:
    HS_MA = HS_MA + ad+ "|"
    
for ad in df_VT["Address"]:
    HS_VT = HS_VT + ad+ "|"
    
for ad in df_CT["Address"]:
    HS_CT = HS_CT + ad+ "|"
    
for ad in df_RI["Address"]:
    HS_RI = HS_RI + ad+ "|"
    
HS_MA = HS_MA[0:-1]
HS_VT = HS_VT[0:-1]
HS_CT = HS_CT[0:-1]
HS_RI = HS_RI[0:-1]

In [0]:
collegeaddstringMA = ""
collegeaddstringNH = ""
collegeaddstringVT = ""
collegeaddstringME = ""
collegeaddstringRI = ""
collegeaddstringCT = ""

#splits the colleges into their respective states and makes them into the pipe separated strings

for ind in range(len(df_colleges_unique["College"])):
    if "Massachussets" in df_colleges_unique["City"][ind]:
        collegeaddstringMA = collegeaddstringMA + df_colleges_unique["College"][ind]+ "|"
    elif "New Hampshire" in df_colleges_unique["City"][ind]:
        collegeaddstringNH = collegeaddstringNH + df_colleges_unique["College"][ind]+ "|"
    elif "Vermont" in df_colleges_unique["City"][ind]:
        collegeaddstringVT = collegeaddstringVT + df_colleges_unique["College"][ind]+ "|"
    elif "Maine" in df_colleges_unique["City"][ind]:
        collegeaddstringME = collegeaddstringME + df_colleges_unique["College"][ind]+ "|"
    elif "Rhode Island" in df_colleges_unique["City"][ind]:
        collegeaddstringRI = collegeaddstringRI + df_colleges_unique["College"][ind]+ "|"
    elif "Connecticut" in df_colleges_unique["City"][ind]:
        collegeaddstringCT = collegeaddstringCT + df_colleges_unique["College"][ind]+ "|"
    
collegeaddstringMA = collegeaddstringMA[0:-1]
collegeaddstringNH = collegeaddstringNH[0:-1]
collegeaddstringVT = collegeaddstringVT[0:-1]
collegeaddstringME = collegeaddstringME[0:-1]
collegeaddstringRI = collegeaddstringRI[0:-1]
collegeaddstringCT = collegeaddstringCT[0:-1]
    
collegeaddstringMA